In [1]:
# !pip install opencv-python==4.5.5.62
# !pip install -U opencv-python

In [2]:
import face_recognition
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import openpyxl
import datetime


# # from IPython.display import display, Javascript, Image
# import PIL
# import io
# import time

In [3]:
path = "train/"

images = []

classnames = []

myList = os.listdir(path=path)


In [4]:
for img in myList:
    currentImg = cv2.imread(filename=f"{path}/{img}")
    images.append(currentImg)
    classnames.append(os.path.splitext(img)[0])

print(classnames)

['Anik', 'Ferdous', 'Harun', 'Masfiq', 'Nibir', 'Niloy', 'Ratul', 'Shafat', 'Shuvo', 'Tanvir', 'Zubair']


In [5]:
def findEncodings(images):
    encodeList = []

    for img in images:
        img = cv2.cvtColor(src=img, code=cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(face_image=img)[0]
        encodeList.append(encode)
    return encodeList

In [6]:
xlsx_path = "attendence.xlsx"
wb = openpyxl.load_workbook(filename=xlsx_path)
sheet=wb.active
wb.sheetnames

['2023-05-29']

In [7]:
def takeAttendance(attendence):
    
    global index
    
    xlsx_path = "attendence.xlsx"
    wb = openpyxl.load_workbook(filename=xlsx_path)
    sheet=wb.active
    
    if str(datetime.datetime.now()).split()[0] in wb.sheetnames:
        sheet=wb.get_sheet_by_name(str(datetime.datetime.now()).split()[0])
    else:
        sheet=wb.create_sheet(title= str(datetime.datetime.now()).split()[0])

    index = 1
    
    for name in attendence:
        attend_date, attend_time = str(datetime.datetime.now()).split()
        sheet.cell(row=index, column=1, value=name)
        sheet.cell(row=index, column=2, value=attend_date)
        sheet.cell(row=index, column=3, value=attend_time[:8])
        wb.save(filename=xlsx_path)
        index += 1

In [8]:
# This is a demo of running face recognition on live video from your webcam. It's a little more complicated than the
# other example, but it includes some basic performance tweaks to make things run a lot faster:
#   1. Process each video frame at 1/4 resolution (though still display it at full resolution)
#   2. Only detect faces in every other frame of video.

# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

from pathlib import Path

today_directory=Path('Attendence_img/'+str(datetime.datetime.now()).split()[0])

if today_directory.exists:
    None
else:
    os.mkdir(today_directory)
# print(str(directory))

# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)

# Create arrays of known face encodings and their names
known_face_encodings = findEncodings(images)
known_face_names = classnames

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

attendence=[]

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Only process every other frame of video to save time
    if process_this_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]
        
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)
            
            
            
            if (name in attendence) or (name =="Unknown"):
                None
            else: 
                attendence.append(name)

    process_this_frame = not process_this_frame
    
         
    

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
        
        if name =="Unknown":
            None
        else: 
            face=frame[top:bottom, left:right]
            cv2.imwrite(os.path.join('Attendence_img/'+str(datetime.datetime.now()).split()[0], f'{name}.jpg'), face)

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

takeAttendance(attendence)

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()